### Dependecies and necessary tools

In [1]:
#Importing dependencies
import pandas as pd
from sodapy import Socrata
import scipy.stats as st
import numpy as np
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager

# Importing data for API call
from config1 import app_token
from config1 import username
from config1 import password

ModuleNotFoundError: No module named 'config1'

### EXTRACT 

In [ ]:
# Getting the data from API NY Open Data for the first database: 
client = Socrata("data.cityofnewyork.us",
                  app_token, username,
                  password)
# Database returned as JSON from API / converted to Python list of dictionaries by sodapy.
results = client.get("gaq9-z3hz", limit=2832)

# Convert to pandas DataFrame
Recycling_Div_Captures = pd.DataFrame.from_records(results)
Recycling_Div_Captures

In [ ]:
# creating path for the citywide subsort 
data_path = "Resources/Citywide_Subsort.csv"
# Read the data into df
cws_df = pd.read_csv(data_path)
#Preview data
cws_df.head()

In [ ]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

In [ ]:
url = 'https://data.cityofnewyork.us/City-Government/DSNY-Waste-Characterization-Mainsort/k3ks-jzek'
browser.visit(url)

In [ ]:
df_list = []
for x in range(1,41):
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    
    table = soup.find_all('div', class_='socrata-table frozen-columns')
    df_list.append(pd.read_html(str(table))[0])
    button = browser.find_by_xpath('//*[@id="app"]/div/div[2]/section[4]/div[2]/div/div/div[4]/div[3]/span[1]/button[2]').click()

In [ ]:
df_combined = pd.concat(df_list)

In [ ]:
df_combined.reset_index(drop=True, inplace=True)

In [ ]:
cwm_df = df_combined
cwm_df.head()

### TRANSFORM

In [ ]:
# Analizing material groups
unique_mat_subsort = cws_df["Material Group"].unique()
print(f'Material groups (subsort): {unique_mat_subsort}')
unique_mat_mainsort = cwm_df["Material Group"].unique()
print(f'Material groups (mainsort): {unique_mat_mainsort}')

In the general database (Recycling_Div_Captures) is possible to see that there is just data for paper and MGP, so from this dataset information just for those materials will be taken. 
The materials will be group in the next collections for a non-relational dabatase:
- Paper
- MGP (Metal, Glass, and Plastic)
- Organic
- E- Waste
- Special Waste
- Others

In [ ]:
unique_loc_subsort = cws_df["Location"].unique()
print(f"Locations register in the Subsort dataset: {unique_loc_subsort}")
print("----------------------------------------------------------------")
unique_loc_mainsort = cwm_df["Location"].unique()
print(f"Locations register in the Mainsort dataset: {unique_loc_mainsort}")
print("----------------------------------------------------------------")
uniqie_loc_general = Recycling_Div_Captures["_zone"].unique()
print(f"Locations register in the general dataset: {uniqie_loc_general}")

Analaizing the locations in all the databases, we decide to summarize and filtering them and their data just for the 5 boroughs.
- Manhattan
- Queens (which includes 'Queens East' and 'Queens West')
- Brooklyn (which includes 'Brooklyn North' and 'Brooklyn South')
- Bronx
- Staten Island


Each material collection will have some next fields per location (materials like paper and MGP will have average capture rates taken from the general dataset, but the other not because we do not have that information):
- Total aggregate percent
- Total refuse percent
- Average capture rate
- Average percent misclassified material 

In [ ]:
# Subsort percentages format 
cols = ["Aggregate Percent", "Refuse Percent", "MGP Percent", "Paper Percent", "Organic Percent"]
cws_df[cols] = cws_df[cols] *100
cws_df.head()

In [ ]:
# # Mainsort percentages format 
# cols = ["Aggregate Percent", "Refuse Percent", "MGP Percent", "Paper Percent", "Organic Percent"]
# cwm_df["Aggregate Percent"][0]


In [ ]:
# First filter: Recycling_Div_Captures data should be filter just for 2017 year
Recycling_Div_Captures["fiscal_year"] = Recycling_Div_Captures["fiscal_year"].apply(pd.to_numeric, errors='coerce')
Recycling_Div_Captures = Recycling_Div_Captures.loc[Recycling_Div_Captures["fiscal_year"]==2017,:]
